# Let's do some biomechanics !!


Note: To run a cell, you press SHIFT+ENTER


## Load a previously created bioMod file
This first section load a bioMod file and allow you to vizualise it if wanted.

In [1]:
# So first, let's import all the required modules 
import biorbd  # This is the core that will do all the calculation
import bioviz  # This is to show the model

import numpy  # Numpy is a python module that helps dealing with the mathematics of matrices and vectors

In [36]:
# Import and interpret the bioMod of the full body
model = biorbd.Model("models/SimpleBody.bioMod")

# Define some useful information
n_q = model.nbQ()  # The number of degrees of freedom in the model

In [18]:
# Run this cell to show the model. 
# Please note that due to some jupyter limitations, you may have to restart after running this cell

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(loaded_model=model)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.set_q([0, 0, -0.15, 1.20, 0.7, -1, 0.45])

# Halt the program so you can interact with the vizualiser. Closing the window should allow to continue
viz.exec()

## Creating some data
The next section generates data. 
This is to simulate collecting data on a real human.
The movement we simulate here is simply raising the arm vertical point down to horizontal pointing forward

The strategy is to create a collection of position (stored in a matrix of "n degrees-of-freedom" by "n frames").
Then to replace some of the data points (namely those related to the arm) so it performs the required movement.
Finally, we will find the position of the skin markers for each of the data points (This is as if you were moving the avatar at a position, and the write down the 3d position of each blue dots). 

Remember that the arm in the SimpleBody.bioMod is the 4th degree-of-freedom so it is indexed "3" as the first index is "0".
The downward pointing is when $q[3] = 0.15$ radian and fronward point at $1.7$ radian. 
We simulate 100 frames in all


In [3]:
# Let's declare some variables to make our life easy
number_frames = 100
starting_arm_position = 0.15
end_arm_position = 1.7

# First create a static body at each time frame
# So let's have the full body at a relaxed position
# Note: the transpose ensures that it is a column vector (time will be on each column)
full_body_position = numpy.array([[0, 0, -0.15, starting_arm_position, 0.7, -1, 0.45]]).transpose()

# Repeat that full body position, one for each frame
full_body_all_position = numpy.repeat(full_body_position, number_frames, axis=1)  # axis=1 is to repeat on columns

In [4]:
# Now we can create an arm movement over time
arm_position_over_time = numpy.linspace(starting_arm_position, end_arm_position, number_frames)

# And replace the arm position in the body positions 
full_body_all_position[3, :] = arm_position_over_time

In [22]:
# If you want to see the movement you just simulated, you can run this cell
# Note: again due to Jupyter limitations, this may or may not cause issue

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(loaded_model=model)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.load_movement(full_body_all_position)

# Halt the program so you can admire the movement with the vizualiser. Closing the window should allow to continue
viz.exec()

In [5]:
# Now we can "collect" the position of each skin markers at each frame
all_skin_markers = []
for f in range(number_frames):
    # Get the skin marker at a position
    skin_markers = list(model.markers(full_body_all_position[:, f]))
    
    # The next for loop goes over all the skin markers and transform them in vector 
    # This is mandatory because of the way biorbd stores data
    for i in range(len(skin_markers)):
        skin_markers[i] = skin_markers[i].to_array()
    
    # Put all these skin marker of a frame in a 3 by "n markers" matrix (3 for x, y and z components)
    skin_markers = numpy.array(skin_markers).transpose()
        
    # Store the results in the main data list
    all_skin_markers.append(skin_markers)

## Inverse kinematics
Inverse kinematics tries to find what are the positions that could have produced a set of given markers
In our case, these markers are the simulated one. 
In the real life, it would be collected data

The stategy is to simulate a lot a different position and find the one that reduces the most the error between the measured markers with the simulated one. Then to repeat for each frame.

In [72]:
# First let's define a function that takes in a position (q) and spits out a matrix of markers (3 x nMarkers)
def forwardKinematics(q):
    markers = list(model.markers(q))
    
    # Put these value in a single vector by concatenating (stacking top/down)
    markersVector = numpy.ndarray((0,))
    for i in range(len(markers)):
        markersVector = numpy.concatenate((markersVector, markers[i].to_array()), axis=0)

    # Return the markers matrix to caller (newaxis converts the vector into a matrix with one column)
    return markersVector[:, numpy.newaxis]

# Define also a jacobian function
def jacobian(q):
    # Now test a bunch of positions to find the best (which minimizes the error with real markers)
    jacobian = list(model.markersJacobian(q))

    # Put these value in a matrix by concatenating (stacking top/down)
    jacobianMatrix = numpy.ndarray((0, n_q))
    for i in range(len(jacobian)):
        jacobianMatrix = numpy.concatenate((jacobianMatrix, jacobian[i].to_array()), axis=0)
    
    # Return the jacobian matrix to caller
    return jacobianMatrix

In [73]:
# The let's test these functions and print the result to make sure it works as expected
q = numpy.array([0, 0, -0.15, 1.7, 0.7, -1, 0.45])
print(f"The markers when the body is at {q} is:")
print("")
print(forwardKinematics(q))
print("")
print("The jacobian matrix at the same position is:")
print(jacobian(q))

The markers when the body is at [ 0.    0.   -0.15  1.7   0.7  -1.    0.45] is:

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.05529211]
 [ 0.3658453 ]
 [ 0.        ]
 [ 0.08219097]
 [ 0.54382409]
 [ 0.        ]
 [ 0.46520345]
 [ 0.35731942]
 [ 0.        ]
 [ 0.15680617]
 [-0.25575736]
 [ 0.        ]
 [ 0.02631651]
 [-0.52589149]
 [ 0.        ]
 [ 0.16631651]
 [-0.52589149]]

The jacobian matrix at the same position is:
[[ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 1.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.          1.          0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.        ]
 [ 1.          0.         -0.3658453   0.          0.          0.
   0.        ]
 [ 0.          1.          0.05529211  0.          0.          0.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.

In [105]:
# Perform the inverse kinematics for the frame 0
real_markers = all_skin_markers[0].transpose().flatten()[:, numpy.newaxis]  # Get the position of real markers in a single vector
number_of_passage = 50  # Number of Newton iterations
q_init = numpy.zeros((n_q, 1))

for i in range(number_of_passage):
    simulated_markers = forwardKinematics(q_init[:, 0])
    jacobianMatrix = jacobian(q_init[:, 0])
    q_init = jacobianMatrix.inverse() @ (simulated_markers - real_markers)  # @ is a matrix multiplication
    # print(q_advance)
    # q_init = q_advance  # Advance the q_init and repeat
print(q_init)

AttributeError: 'numpy.ndarray' object has no attribute 'inverse'

(21, 1)

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.05529211,
        0.3658453 ,  0.        ,  0.08219097,  0.54382409,  0.        ,
        0.05529211, -0.0441547 ,  0.        ,  0.15680617, -0.25575736,
        0.        ,  0.02631651, -0.52589149,  0.        ,  0.16631651,
       -0.52589149])

(21, 7)

(21, 1)